In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
'''
import os
import glob
import random
import shutil

import librosa
import soundfile as sf
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

#from pydub import AudioSegment
'''

In [ ]:
!ls

sample_data


In [ ]:
!unzip "/gdrive/MyDrive/JWick/gun_shot_wav.zip" -d "/content"

In [ ]:
!unzip "/gdrive/MyDrive/JWick/backgound_2_seconds.zip" -d "./"

In [ ]:
# folder path
dir_path = '/content/content/gun_shot_wav'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 260


In [ ]:
def show_sample(audio_path):
  audio_data, sample_rate = sf.read(audio_path)
  class_name = os.path.basename(os.path.dirname(audio_path))
  print(f'Class: {class_name}')
  print(f'File: {audio_path}')
  print(f'Sample rate: {sample_rate}')
  print(f'Sample length: {len(audio_data)}')

  plt.title(class_name)
  plt.plot(audio_data)
  display(Audio(audio_data.T, rate=sample_rate))

In [ ]:
show_sample("/content/content/background/silence3.wav")

In [ ]:
!mkdir dataset
%cd dataset
!mkdir train test
%cd train
!mkdir background gunshot
%cd ../
%cd test
!mkdir background gunshot

/content/dataset
/content/dataset/train
/content/dataset
/content/dataset/test


In [ ]:
# move 180 files to train folders
!ls /content/content/background/* | head -180 | xargs -I{} mv {} /content/dataset/train/background
!ls /content/content/gun_shot_wav/* | head -180 | xargs -I{} mv {} /content/dataset/train/gunshot

In [ ]:
!ls /content/content/background/* | head -20 | xargs -I{} mv {} /content/dataset/test/background
!ls /content/content/gun_shot_wav/* | head -20 | xargs -I{} mv {} /content/dataset/test/gunshot

In [ ]:
%cd ../../

/content


Installing

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!./Miniconda3-py39_23.3.1-0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages')

In [ ]:
!conda create -n myenv python=3.9

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv && pip install tflite-model-maker && sudo apt -y install libportaudio2 && pip install numpy==1.23.4 && pip install ipykernel && pip install seaborn

In [ ]:
%%writefile train_model_script.py

import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random
import IPython

from IPython.display import Audio, Image
from IPython import display
from scipy.io import wavfile

####################################
# Directory
data_dir = './dataset'

#Specs
################################################################################       spec = audio_classifier.BrowserFftSpec()
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=1 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=2 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

####################################
# Data
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)

####################################
# Train
batch_size = 128
epochs = 20

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

#####################################
# export
models_path = './models'
print(f'Exporing the TFLite model to {models_path}')
model.export(models_path, tflite_filename='gunshot_model.tflite')

#####################################
# evaluate
print('Evaluating the model')
model.evaluate(test_data)

#####################################
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

#####################################

Overwriting train_model_script.py


In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python train_model_script.py